In [2]:
from keras.models import load_model
import keras
import tensorflow as tf


In [3]:
%run ../../layers/dimension_utils

In [11]:
@keras.saving.register_keras_serializable()
class TFReduceSum(keras.layers.Layer):
    def __init__(self, tensor_grap, node_weights, node_inputs, node_attribute, *args, **kwargs):
        super().__init__()
        self.tensor_grap = tensor_grap
        self.node_weights = node_weights
        self.node_inputs = node_inputs
        self.node_attribute = node_attribute
        self.keep_dims = node_attribute.get("keepdims", 1) == 1
        # print('tennn: ', tensor_grap[node_inputs[0]])
        input_shape_len = len(tensor_grap[node_inputs[0]]['config']['shape'])
        # channel_to_last_dimension is from dimension_utils
        self.axes = [channel_to_last_dimension(i) if i >=0 else channel_to_last_dimension(input_shape_len + i) for i in node_attribute.get("axes", [-1])]

    def call(self, inputs, *args, **kwargs):
        return keras.ops.sum(inputs, axis = self.axes, keepdims=self.keep_dims)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "tensor_grap":self.tensor_grap,
            'node_weights':self.node_weights,
            'node_inputs':self.node_inputs,
            'node_attribute':self.node_attribute
        })
        return config

class TFLog(keras.layers.Layer):
    def __init__(self, *args, **kwargs):
        super().__init__()
    def call(self, inputs, *args, **kwargs):
        return keras.ops.log(inputs)

from keras import backend as K
K.clear_session()
keras.saving.get_custom_objects().clear()
custom_objects = {"TFReduceSum": TFReduceSum, 'TFLog': TFLog}
with keras.saving.custom_object_scope(custom_objects):
    model = load_model("sum(log).keras")
    model.summary()

print(model.layers)


node_inputs:  <class 'dict'>


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (1, 3, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tf_log (TFLog)                  │ (1, 3, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tf_reduce_sum (TFReduceSum)     │ (1, 1)                 │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

[<InputLayer name=input_layer, built=True>, <TFLog name=tf_log, built=True>, <TFReduceSum name=tf_reduce_sum, built=True>]


In [8]:
import torch
_input = torch.tensor([10, 40, 50], dtype = torch.float32).reshape(1,-1,1) 
model.predict(_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


array([[9.903488]], dtype=float32)